In [20]:
# Set up enviroment
import json
import re
import pandas as pd
import random
from dotenv import load_dotenv
from llama_index.core import (
    Document,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
    get_response_synthesizer,
)
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.evaluation import RetrieverEvaluator
from llama_index.core.prompts import PromptTemplate
from llama_index.core.schema import QueryBundle
from llama_index.core.settings import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# Key
load_dotenv()
# Set embedding model
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# Accuracy
tasa_acierto = True
test_exam = 2

In [2]:
# Carga de las preguntas
qa_path = "../data/exams_test2/clean_questions/questions_all_tests.json"
with open(qa_path, "r", encoding="utf-8") as f:
    questions = json.load(f)

In [3]:
# Ruta al índice optimizado
storage_path = "../data/index_storage/"
storage_context = StorageContext.from_defaults(persist_dir=storage_path)
index = load_index_from_storage(storage_context)

# Configurar el retriever
retriever = index.as_retriever(similarity_top_k=5)

In [4]:
# Función auxiliar para formatear las opciones:
def format_options(options_dict):
    clean_options = {}
    for key in options_dict:
        # Elimina letras errantes como '\nB' o '\nC' dentro del texto
        value = options_dict[key]
        value = re.sub(r'\n[A-E]\n?', '', value).strip()
        clean_options[key] = value
    return "\n".join([f"{k}. {v}" for k, v in clean_options.items()])

# Validar la repsuesta:
def display_question_result(result):
    print("ID Pregunta:", result["question_id"])
    print("\nPregunta:\n", result["question"])
    print("\nOpciones:\n", result["options"])
    print("\nAcertó?:", result["is_correct"])
    print("\nRespuesta generada por el modelo:\n", result["response_obj"])
    print("\nRespuesta correcta:", result["correct_answer"])
    if "response_obj" in result:
        print(f"\n🔎 Chunks usados para responder la pregunta {result['question_id']}:\n")
        for i, node in enumerate(result["response_obj"].source_nodes):
            print(f"--- Chunk #{i+1} | Score: {node.score:.4f} ---")
            print(node.node.get_content().strip())
            print()

system_prompt = """
Eres un experto en Apache Spark y PySpark, y estás respondiendo preguntas tipo test usando exclusivamente la información del contexto proporcionado.

No puedes utilizar ningún conocimiento previo ni información externa al contexto.

Tu tarea es:
1. Leer el contexto cuidadosamente.
2. Leer la pregunta y las opciones tipo test.
3. Seleccionar la opción correcta si está claramente justificada por el contenido del contexto, y justificar tu elección citando fragmentos textuales exactos.
4. Si no hay una justificación directa, puedes razonar una respuesta **únicamente si se basa en información presente en el contexto**, indicando exactamente **qué fragmentos usas** y **cómo los conectas**. No está permitido inventar datos.
5. En caso de no encontrar ninguna información útil en el contexto, responde exactamente con: **"No existe información"**
"""

llm = OpenAI(
    model="gpt-4o-mini",
    temperature=0,
    system_prompt=system_prompt
)

Settings.llm = llm

def extract_predicted_letter(response_text):
    """
    Extrae la letra (A, B, C, D o E) de una respuesta generada en el formato:
    'Respuesta correcta: <LETRA>\nJustificación: <...>'
    """
    match = re.search(r"Respuesta\s+correcta\s*:\s*([A-E])", response_text, flags=re.IGNORECASE)
    if match:
        return match.group(1).upper()
    return None 

def ask_question_with_options(question_obj, 
                              #query_str_template, 
                              retriever = retriever, response_mode= "compact"):
    question_text = question_obj["question"]
    options_text = format_options(question_obj["options"])

    #qa_prompt = PromptTemplate(query_str_template)
    #Settings.text_qa_template = qa_prompt

    query_str = f"""
    {question_text}

    Opciones: 
    {options_text}
        
    IMPORTANTE:
        - Usa solo información del contexto.
        - No respondas con conocimientos externos.
        - Aunque sepas la respuesta correcta, si no existe esa información en el contexto devuelve "No existe información"
    
    Responde de la siguiente manera:
    "
    Respuesta correcta: <LETRA>
    Justificación: <JUSTIFICACIÓN>
    "
    """
    
    # Usar QueryBundle para pasar la información
    query_bundle = QueryBundle(
        query_str = query_str,
        custom_embedding_strs = [question_text, options_text]
    )

    response_synthesizer = get_response_synthesizer(response_mode=response_mode)
    
    query_engine = RetrieverQueryEngine(
        retriever=retriever,
        response_synthesizer=response_synthesizer
    )
    
    response = query_engine.query(query_bundle)
    predicted_answer = response.response.strip()
    
    return {
        "question_id": question_obj["question_id"],
        "question" : question_obj["question"],
        "options": format_options(question_obj["options"]),
        "predicted_answer": predicted_answer,
        "correct_answer": question_obj["correct_answer"],
        "is_correct": extract_predicted_letter(predicted_answer) == question_obj["correct_answer"],
        "response_obj": response
    }

In [9]:
questions_to_evaluate = random.sample(questions, 5)

In [6]:
for q in questions_to_evaluate:
    response = ask_question_with_options(q, response_mode = "tree_summarize")
    display_question_result(response)

ID Pregunta: Test_2_13

Pregunta:
 What is the core of Spark’s fault-tolerant mechanism?

Opciones:
 A. RDD is at the core of Spark, which is fault tolerant by design
B. Data partitions, since data can be recomputed
C. DataFrame is at the core of Spark since it is immutable
D. Executors ensure that Spark remains fault tolerant

Acertó?: True

Respuesta generada por el modelo:
 Respuesta correcta: A  
Justificación: "RDDs serve as the core data structure in Spark, enabling fault-tolerant and parallel operations on large-scale distributed datasets and they are immutable." Además, se menciona que "Spark is designed to be fault-tolerant," lo que refuerza que RDD es fundamental para este mecanismo.

Respuesta correcta: A

🔎 Chunks usados para responder la pregunta Test_2_13:

--- Chunk #1 | Score: 0.7342 ---
RDDs are fault-tolerant. This means that if there are failures, RDDs have the ability to self-recover. 
Spark achieves that by distributing these RDDs to different worker nodes while ke

## Implementación de extract_predicted_letter con `pydantic`

#### Objetivo

Sustituir la función tradicional `extract_predicted_letter`, basada en expresiones regulares, por un mecanismo robusto y validado mediante el modelo `pydantic.BaseModel`.

Este modelo permite extraer y validar automáticamente los dos campos clave generados por el modelo LLM:

+   respuesta_correcta: la letra elegida como respuesta
+   justificacion: explicación textual del porqué

#### Modelo de validación estructural

In [40]:
from pydantic import BaseModel, Field

class QuestionAnswer(BaseModel):
    respuesta_correcta: str = Field(
        ...,
        pattern="^[A-E]$",
        description="Letra correspondiente a la opción seleccionada por el modelo como respuesta correcta. Debe ser una letra entre A y E."
    )
    justificacion: str = Field(
        ...,
        description="Explicación textual que justifica por qué se ha seleccionado la respuesta correcta, basándose únicamente en el contexto proporcionado."
    )

Este modelo reemplaza la lógica basada en re.search(...) y permite validación tipada, restringiendo la salida del modelo a formatos esperados.

#### Nueva integración con `ask_question_with_options`

En la función de evaluación de preguntas tipo test, se actualizó la configuración del `response_synthesizer` para que utilice el modelo `QuestionAnswer`:

```
response_synthesizer = get_response_synthesizer(
    response_mode=response_mode,
    output_cls=QuestionAnswer
)
```

Esto fuerza al motor de consulta a intentar parsear la respuesta generada directamente como una instancia válida de `QuestionAnswer`.

In [46]:
def ask_question_with_options(question_obj, 
                              #query_str_template, 
                              retriever = retriever, response_mode= "compact"):
    question_text = question_obj["question"]
    options_text = format_options(question_obj["options"])

    #qa_prompt = PromptTemplate(query_str_template)
    #Settings.text_qa_template = qa_prompt

    query_str = f"""
    {question_text}

    Opciones: 
    {options_text}
        
    IMPORTANTE:
        - Usa solo información del contexto.
        - No respondas con conocimientos externos.
        - Aunque sepas la respuesta correcta, si no existe esa información en el contexto devuelve "No existe información"
    
    Responde de la siguiente manera:
    Respuesta correcta: <LETRA>
    Justificación: <JUSTIFICACIÓN>
    """
    
    # Usar QueryBundle para pasar la información
    query_bundle = QueryBundle(
        query_str = query_str,
        custom_embedding_strs = [question_text, options_text]
    )

    response_synthesizer = get_response_synthesizer(
        response_mode=response_mode,
        output_cls=QuestionAnswer
    )
    
    query_engine = RetrieverQueryEngine(
        retriever=retriever,
        response_synthesizer=response_synthesizer
    )
    
    response = query_engine.query(query_bundle)
    # predicted_answer = response.response.strip()

    print("🧪 Texto bruto generado por el modelo:\n")
    print(response.response)

    parsed = response.response

    if parsed is None:
        print("\n❌ No se pudo parsear la respuesta con el modelo Pydantic.")
    else:
        print("\n✅ Letra extraída:", parsed.respuesta_correcta)
        print("🧠 Justificación:", parsed.justificacion)

    
    return {
        "question_id": question_obj["question_id"],
        "question" : question_obj["question"],
        "options": format_options(question_obj["options"]),
        "predicted_answer": parsed.respuesta_correcta,
        "correct_answer": question_obj["correct_answer"],
        "is_correct": (parsed.respuesta_correcta == question_obj["correct_answer"]) if parsed else False,
        "response_obj": response
    }

In [47]:
response = ask_question_with_options(questions_to_evaluate[0], response_mode = "tree_summarize")
display_question_result(response)

🧪 Texto bruto generado por el modelo:

respuesta_correcta='A' justificacion='AQE collects runtime statistics, such as data size, skewness, and partitioning, during query execution.'

✅ Letra extraída: A
🧠 Justificación: AQE collects runtime statistics, such as data size, skewness, and partitioning, during query execution.
ID Pregunta: Test_2_3

Pregunta:
 Which of the following is one of the tasks of Adaptive Query Execution in Spark?

Opciones:
 A. Adaptive Query Execution collects runtime statistics during query execution to optimize 
query plans
B. Adaptive Query Execution is responsible for distributing tasks to executors
C. Adaptive Query Execution is responsible for wide operations in Spark
D. Adaptive Query Execution is responsible for fault tolerance in Spark

Acertó?: True

Respuesta generada por el modelo:
 {"respuesta_correcta":"A","justificacion":"AQE collects runtime statistics, such as data size, skewness, and partitioning, during query execution."}

Respuesta correcta: A

# Tasa de acierto

Se le enfrenta a un Test completo, por ejemplo el Test 1, y se calculará el porcentaje de acierto del modelo. 

In [7]:
if tasa_acierto:

    # Filtrar preguntas por test (Test_1_ o Test_2_)
    prefix = f"Test_{test_exam}_"
    preguntas_test = [q for q in questions if q["question_id"].startswith(prefix)]

    total = len(preguntas_test)
    aciertos = 0
    resultados = []  # Para análisis posterior si se desea

    print(f"\nEjecutando evaluación del Test {test_exam} con {total} preguntas...\n")

    for i, pregunta in enumerate(preguntas_test):
        # print(f"➡️ Pregunta {i+1}/{total} - ID: {pregunta['question_id']}")
        result = ask_question_with_options(pregunta)
        resultados.append(result)

        if result["is_correct"]:
            aciertos += 1
            # print("¡Correcta!\n")
        else:
            # print("Incorrecta\n")

        # Puedes descomentar para ver detalles por pregunta
        # display_question_result(result)

    tasa = aciertos / total
    print(f"\nTasa de acierto en el Test {test_exam}: {tasa:.2%} ({aciertos}/{total})")



Ejecutando evaluación del Test 2 con 60 preguntas...

➡️ Pregunta 1/60 - ID: Test_2_1
¡Correcta!

➡️ Pregunta 2/60 - ID: Test_2_2
¡Correcta!

➡️ Pregunta 3/60 - ID: Test_2_3
¡Correcta!

➡️ Pregunta 4/60 - ID: Test_2_4
¡Correcta!

➡️ Pregunta 5/60 - ID: Test_2_5
¡Correcta!

➡️ Pregunta 6/60 - ID: Test_2_6
¡Correcta!

➡️ Pregunta 7/60 - ID: Test_2_7
¡Correcta!

➡️ Pregunta 8/60 - ID: Test_2_8
¡Correcta!

➡️ Pregunta 9/60 - ID: Test_2_9
Incorrecta

➡️ Pregunta 10/60 - ID: Test_2_10
¡Correcta!

➡️ Pregunta 11/60 - ID: Test_2_11
Incorrecta

➡️ Pregunta 12/60 - ID: Test_2_12
¡Correcta!

➡️ Pregunta 13/60 - ID: Test_2_13
¡Correcta!

➡️ Pregunta 14/60 - ID: Test_2_14
¡Correcta!

➡️ Pregunta 15/60 - ID: Test_2_15
¡Correcta!

➡️ Pregunta 16/60 - ID: Test_2_16
¡Correcta!

➡️ Pregunta 17/60 - ID: Test_2_17
¡Correcta!

➡️ Pregunta 18/60 - ID: Test_2_18
Incorrecta

➡️ Pregunta 19/60 - ID: Test_2_19
¡Correcta!

➡️ Pregunta 20/60 - ID: Test_2_20
¡Correcta!

➡️ Pregunta 21/60 - ID: Test_2_21
Incorrect